<a href="https://colab.research.google.com/github/Krishraj13/git-devops-repository/blob/master/internship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
from sklearn.linear_model import LinearRegression
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
import pandas as pd
df = pd.read_csv("prediction.csv")
df['building_type'] = df['building_type'].astype('category').cat.codes
df['daily_wage_per_worker'] = df['daily_wage_per_worker'].replace('[\$,]', '', regex=True).astype(float)
df['total_daily_wage'] = df['total_daily_wage'].replace('[\$,]', '', regex=True).astype(float)
x = df[['building_type','sft','floors','houses_per_floor','rooms_per_house','office_floor']]
y = df[['bricks','cement_bags','sand_tons','steel_tons','workers_required','total_daily_wage']]
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
base_model = LinearRegression()
multi_output_model = MultiOutputRegressor(base_model)
multi_output_model.fit(x_train, y_train)
print("Names of the building types we use are:\n0: apartment\n1: apartment_with_office_floor\n2: independent_house\n3: office_building")
building_type = int(input("Enter the building type (0-3): "))
sft = int(input("Enter the sft (100-10000): "))
floors = int(input("Enter the number of floors: "))
houses_per_floor = int(input("Enter number of houses per floor(0 if it is independent house): "))
rooms_per_house = int(input("Enter number of rooms per house: "))
office_floor = int(input("Enter number of office floors (0 or 1): "))
features = pd.DataFrame([[building_type, sft, floors, houses_per_floor, rooms_per_house, office_floor]],
                        columns=['building_type','sft','floors','houses_per_floor','rooms_per_house','office_floor'])
prediction = multi_output_model.predict(features)
output_labels = ['bricks', 'cement_bags', 'sand_tons', 'steel_tons', 'workers_required', 'total_daily_wage']
print("\n Predicted Construction Requirements:")
for label, value in zip(output_labels, prediction[0]):
    print(f"{label}: {round(value, 0)}")


Names of the building types we use are:
0: apartment
1: apartment_with_office_floor
2: independent_house
3: office_building
Enter the building type (0-3): 0
Enter the sft (100-10000): 870
Enter the number of floors: 3
Enter number of houses per floor: 1
Enter number of rooms per house: 2
Enter number of office floors (0 or 1): 1

 Predicted Construction Requirements:
bricks: 6090.0
cement_bags: 13.0
sand_tons: 10.0
steel_tons: 2.0
workers_required: 3.0
total_daily_wage: 1345.0
